## 0x01 Basic graph(constant 和 operation)

In [4]:
import tensorflow as tf

# create a node called Operation（Op）
a = tf.constant(5, name="input_a")

# create a node called Operation（Op）
b = tf.constant(3, name="input_b")

# tensorflow 创建节点时已经包含了相应的 Op 用于提供计算需要的输入
# tensorflow 会自动绘制必要的连接
c = tf.multiply(a, b, name="mul_c")
d = tf.add(a, b, name="add_d")

# 这里定义了数据流的终点 e
e = tf.add(c, d, name="add_e")

### 上面定义了若干节点，完成了一个小规模数据流图的完整定义

### Session 对象在运行时负责对数据流图进行监督，并且是运行数据流图的主要接口。

当然，tf.Session 有一个与之十分相近的变体 tf.InteractiveSession。


In [7]:
sess = tf.Session()

# 定义一个字典，比如将 a 的值替换为 15
# replace_dict = {a: 15}
# result = sess.run(e, feed_dict=replace_dict)

result = sess.run(e)

print(result)

# 习惯上会用 tf.summary.FileWriter 保存来自数据流图的数据和概括统计量。
writer = tf.summary.FileWriter("./my_graph", sess.graph)

writer.close()
sess.close()

63


## 0x02 基础张量理解


In [8]:
import tensorflow as tf

f = tf.constant([3, 5], name="input_f")
print(f)

import numpy as np

# 视为 0 阶张量或”标量“
t_0 = 50
print(np.array(t_0).shape)
# 视为 1 阶张量或“向量”
t_1 = ["apple", "peach", "grape"]
print(np.array(t_1).shape)
# 视为 2 阶张量或“矩阵”
t_2 = [
    [True, False, False],
    [True, False, False],
    [True, False, False],
]
print(np.array(t_2).shape)
# 视为 3 阶张量
t_3 = [[[0, 0], [0, 1], [0, 2]],
      [[1, 0], [1, 1], [1, 2]],
      [[2, 0], [2, 1], [2, 2]]]

print(np.array(t_3).shape)
print(np.array(t_3).flags)

Tensor("input_f:0", shape=(2,), dtype=int32)
()
(3,)
(3, 3)
(3, 3, 2)
  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  UPDATEIFCOPY : False


## 0x03 占位符和 variable 对象

0x01 并未使用真正的输入，而是常量。而占位符的作用是为运行时即将到来的某个 tensor 对象预留位置。

In [10]:
import tensorflow as tf
import numpy as np

# 创建一个长度为 2，数据类型为 int32 的占位向量
a = tf.placeholder(tf.int32, shape=[2], name="user_input")

# 将该占位向量视为其他任意 tensor 对象，加以利用
b = tf.reduce_prod(a, name="prod_b")
c = tf.reduce_sum(a, name="sum_c")

# 完成数据流图的定义
d = tf.add(b, c, name="add_d")

sess_placeholder = tf.Session()

input_dict = {a: np.array([5, 3], dtype=np.int32)}

sess_placeholder.run(d, feed_dict=input_dict)

my_var = tf.Variable(3, name="my_var")

add = tf.add(5, my_var)
mul = tf.multiply(8, my_var)

# Variable 的初始化

# 必须在一个 session 对象中对 Variable 对象进行初始化，
# 这样会 session 对象开始追踪这个 variable 对象的值的变化
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

var1 = tf.Variable(0, name="initialize_me")
var2 = tf.Variable(1, name="no_initialize")
init = tf.initialize_all_variables([var1], name="init_var1")
sess = tf.Session()
sess.run(init)


# Variable 的初始化

my_var = tf.Variable(1)

# 创建一个 Op, 使其在每次运行时都将该 variable 对象乘以 2
my_var_times_two = my_var.assign(my_var * 2)

# 自增 1
my_var_plus_one = my_var.assign_add(1)
# 自减 1
my_var_plus_one = my_var.assign_sub(1)

init = tf.initialize_all_variables()
sess = tf.Session()

sess.run(init)
sess.run(my_var_times_two)
sess.run(my_var_plus_one)
sess.run(my_var_plus_one)

# trainable 参数
# 本书后续将介绍各种能够自动训练机器学习模型的 optimizer 类，这意味着这些类将自动修改 variable 对象的值，而无需显式请求
# variable 对象默认 trainable=True, 如果不希望 optimizer 类修改它
# no_trainable = tf.Variable(0, trainable=False)



23

## 0x04 辅助函数

In [15]:
# 2*2 的零矩阵
zeros = tf.zeros([2, 2])

# 长度为 6 的全 1 向量
ones = tf.ones([6])

# 3*3*3 的张量，其元素服从 0～10 的均匀分布
uniform = tf.random_uniform([3, 3, 3], minval=0, maxval=10)

# 3*3*3 的张量，其元素服从0均值，标准差为 2 的正态分布
noraml = tf.random_normal([3, 3, 3], mean=0.0, stddev=2.0)

# 该 tensor 对象不会返回任何小于 3.0 或大于 7.0 的值
trunc = tf.truncated_normal([2, 2], mean=5.0, stddev=1.0)

# 默认均值为 0，默认标准差为 1.0
random_var = tf.Variable(tf.truncated_normal([2, 2]))

## 0x05 通过名称作用域组织数据流图


In [17]:
# Basic
import tensorflow as tf

with tf.name_scope("Scope_A"):
    a = tf.add(1, 2, name="A_add")
    b = tf.multiply(a, 3, name="A_mul")
    
with tf.name_scope("Scope_B"):
    c = tf.add(4, 5, name="B_add")
    d = tf.multiply(c, 6, name="B_mul")
    
e = tf.add(b, d, name="output")

# 习惯上会用 tf.summary.FileWriter 保存来自数据流图的数据和概括统计量。
writer = tf.summary.FileWriter("./name_scope_1_graph", graph=tf.get_default_graph())
writer.close()

In [4]:
import tensorflow as tf

graph = tf.Graph()

with graph.as_default():
    in_1 = tf.placeholder(tf.float32, shape=[], name="input_a")
    in_2 = tf.placeholder(tf.float32, shape=[], name="input_b")
    const = tf.constant(3, dtype=tf.float32, name="static_value")

    with tf.name_scope("Transfomation"):
        with tf.name_scope("A"):
            A_mul = tf.multiply(in_1, const)
            A_out = tf.subtract(A_mul, in_1)
            
        with tf.name_scope("B"):
            B_mul = tf.multiply(in_2, const)
            B_out = tf.subtract(B_mul, in_2)
          
        with tf.name_scope("C"):
            C_div = tf.div(A_out, B_out)
            C_out = tf.add(C_div, const)
        
        with tf.name_scope("D"):
            D_div = tf.div(B_out, A_out)
            D_out = tf.add(D_div, const)
        
    out = tf.maximum(C_out, D_out)

    writer = tf.summary.FileWriter("./name_scope_2_graph", graph=graph)
    writer.close()